In [ ]:
import xml.etree.ElementTree as ET
import typing
import pandas as pd

In [ ]:
# fannys = root.find("channel").findall("item/reporter[@username='fanny.blanc']/..")

In [ ]:
class Commande(typing.NamedTuple):
    title: str
    key: str
    link: str
    projet: str
    tiers: str
    montant: float

In [ ]:
props = {
    "tiers": {"name": "customfield_13225", "fct": lambda x: x},
    "montant": {"name": "customfield_12301", "fct": float},
    "projet-budgetaire": {
        "name": "customfield_11103",
        "fct": lambda x: x,
    },
}


def getter(c, code):
    p = props[code]
    return p["fct"](
        c.find(
            f"customfields/customfield[@id='{p['name']}']/customfieldvalues/customfieldvalue"
        ).text
    )

In [ ]:
def build(node):
    return Commande(
        node.find("title").text,
        node.find("key").text,
        node.find("link").text,
        getter(node, "projet-budgetaire"),
        getter(node, "tiers"),
        getter(node, "montant"),
    )

In [ ]:
def buildDataFrame(file):
    tree = ET.parse(file)
    root = tree.getroot()
    items = root.find("channel").findall("item")
    selection = items
    commande_nodes = [i for i in selection if i.find(".//type").text == "Commande"]
    commandes = [build(c) for c in commande_nodes]
    return pd.DataFrame(commandes)

In [ ]:
df0310 = buildDataFrame("/home/thomas/Téléchargements/masa/export10.03.2025.xml")
df0325 = buildDataFrame("/home/thomas/Téléchargements/masa/SearchRequest20032025.xml")

In [ ]:
df0310

In [ ]:
res = df0310[df0310.key != "CMDBRB-229"].merge(df0325, how="outer", on="key")

In [ ]:
assert res[res.tiers_y != res.tiers_y].size == 0

In [ ]:
res[(~res.title_x.isna()) * (res.montant_x != res.montant_y)]

In [ ]:
res[res.tiers_x.str.startswith("1700000881").fillna(False)]

In [ ]:
res[["tiers_y", "montant_x", "montant_y"]].groupby("tiers_y").sum()

In [ ]:
res.to_excel("res.xlsx")